In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import os
import subprocess
import bibtexparser

In [3]:
ref_wolf = {
    25: 'ARZATE2020110265',
    51: 'Choubert2011-ic', 
    5: 'GAROMA2010814',
    33: 'KATSOYIANNIS20042685',
    31: 'Kim2009-ik',
    32: 'Kruithof2007-os', 
    48: 'Kuang2013-mo',
    9: 'Marcelino2017-wn',
    38: 'Mohamad_Mazuki2020-dr',
    12: 'Javid2020-ch',
    15: 'Ormad2008-bd',
    49: 'Radjenovic2009-ud',
    17: 'RAHMANI201882',
    18: 'Rizzo2019-us',
    35: 'Ruan2019-vy',
    26: 'Salehi2020-aa',
    27: 'Samarghandi2007-uz',
    20: 'Scheideler2011-pu',
    21: 'Shahmahdi2020-ok',
    22: 'Tkachenko2020-sk',
    34: 'Wu2022-sp'}

In [4]:
ref_lluis = {
    
}

In [5]:
cwd = Path(os.path.abspath(''))
cwd

PosixPath('/mnt/c/Users/joans/OneDrive/Escriptori/icra/traca_contaminacio/traca_contaminacio')

In [6]:
def parse_spaces_df(df):
    regex = r'\xa0(?=$)'
    df = df.copy()
    df.replace(regex, '', regex=True, inplace=True)   #detele spaces at end of word
    df.replace('\xa0', ' ', regex=True, inplace=True) #replace remaining \xa0 with space
    return df

def parse_array(arr):
    result = []
    arr = filter(lambda x: x != ',', arr)
    for item in arr:
        if ',' in item:
            # If the item contains commas, split and convert to integers
            result.extend([int(x) for x in item.split(',')])
        else:
            # If it's just a single number, convert it to integer
            result.append(int(item))
    return result

def parse_df(df):
    df = df.copy()
    df['Referència'] = df['Referència'].astype(str)
    df = df[df['Referència'] != '-\xa0'].copy()
    df['Referència'] = df['Referència'].str.replace('.00', '')
    df['Referència'] = df['Referència'].str.replace('.0', '')
    df['Referència'] = df['Referència'].str.replace('.', ',')
    df['Referència'] = df['Referència'].str.replace('-', ',')
    df = df[df['Referència'] != ','].copy()

    return df

def add_cites(df, refs_dict):
    df = parse_df(df).copy()
    new_cites = []

    for index, row in df.iterrows():
        
        #split by comma
        refs = row['Referència'].split(',')
        poll = row['Component']

        for ref in refs:
            int_ref = int(ref)
            if int_ref in ref_wolf:
                cite = ref_wolf[int(ref)]
                if poll not in refs_dict:
                    refs_dict[poll] = set()
                refs_dict[poll].add(cite)

def get_references_of_df(df):
    df_parsed = parse_df(df)
    return set(parse_array(df_parsed[['Referència']].values.flatten()))



In [14]:
atenuacio_df = pd.read_excel(cwd / 'data' / 'atenuacions_generacions.xlsx').drop(columns=['Error industrial'])
compounds_df = pd.read_excel(cwd / 'data' / 'compound_features.xlsx').dropna()
traduccio_df = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx')
sf = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx', sheet_name='SF')[['Component', 'Referència']].dropna()
o3 = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx', sheet_name='O3')[['Component', 'Referència']].dropna()
gac = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx', sheet_name='GAC')[['Component', 'Referència']].dropna()
uv = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx', sheet_name='UV')[['Component', 'Referència']].dropna()
uf = pd.read_excel(cwd / 'data' / 'cat_english_pollutant_translation.xlsx', sheet_name='UF')[['Component', 'Referència']].dropna()
review = pd.read_excel(cwd / 'data' / 'Review.xlsx')

In [15]:
atenuacio_df = parse_spaces_df(atenuacio_df)
compounds_df = parse_spaces_df(compounds_df)
traduccio_df = parse_spaces_df(traduccio_df)
sf = parse_spaces_df(sf)
o3 = parse_spaces_df(o3)
gac = parse_spaces_df(gac)
uv = parse_spaces_df(uv)
uf = parse_spaces_df(uf)

In [9]:
get_references_of_df(sf).union(get_references_of_df(o3)).union(get_references_of_df(gac)).union(get_references_of_df(uv)).union(get_references_of_df(uf))

{5,
 9,
 12,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 25,
 26,
 27,
 28,
 31,
 32,
 33,
 34,
 35,
 36,
 48,
 49,
 51}

In [10]:
review = review[['Pub doi', 'Nom C']]
review = review.dropna().copy()
review = review[review['Pub doi'] != '-\xa0']
review = review[review['Pub doi'] != '-']
review['Pub doi'] = review['Pub doi'].str.replace('\xa0', '')
review = review.dropna()

In [11]:
review

,Pub doi,Nom C
0,https://doi.org/10.2175/106143096X127695,1.1.1-Tricloroetà
1,https://doi.org/10.1016/j.seppur.2006.05.013,1.1.1-Tricloroetà
2,https://doi.org/10.2175/106143096X127695,1.1.1-Tricloroetà
3,https://doi.org/10.1016/j.chemosphere.2018.11.200,1.1.1-Tricloroetà
8,http://www.jstor.org/stable/25042837,1.2-Dicloroetà
...,...,...
1204,https://sci-hub.se/10.3390/w10091248,Zinc dissolt
1205,https://sci-hub.se/10.1080/00139307309435517,Zinc dissolt
1206,https://sci-hub.se/10.1065/espr2006.10.355,Zinc dissolt
1207,https://doi.org/10.1007/s11356-018-1915-4,Zinc dissolt


In [12]:
#store list of references to txt
with open(cwd / 'data' / 'references.txt', 'w') as f:
    for item in set(review[['Pub doi']].values.flatten()):
        f.write("%s\n" % item)

ref_pd = review.filter(items=['Pub doi', 'Nom C']).drop_duplicates()


def fetch_bibtex(doi):
    try:
        # Run the doi2bib command in the terminal
        result = subprocess.run(['doi2bib', doi], capture_output=True, text=True, check=True)
        
        if result.stdout == '':
            return None
        
        # Extract the BibTeX entry from the result
        bibtex_entry = result.stdout.strip()
        return bibtex_entry
    except subprocess.CalledProcessError as e:
        print("Error:", e.stderr)
        return None



correct_links = []

for row in ref_pd.itertuples():
    doi = row[1]
    contaminant = row[2]
    fetch = fetch_bibtex(doi)
    if fetch is not None:
        correct_links.append({
            'doi': doi,
            'bibtex': fetch,
            'contaminant': contaminant
        })
    else:
        print('Error fetching doi:', doi)

Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: doi.org/10.1016/ S1473-3099(20)30332-7
Error fetching doi: http://www.jstor.org/stable/25039577
Error fetching doi: Applying life cycle assessment to assess the environmental performance of decentralised versus centralised wastewater systems
Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: https://www.jstor.org/stable/25038362
Error fetching doi: https://sci-hub.se/10.2307/25038362
Error fetching doi: https://doi.org/10.1007/s11356-018-1915-5
Error fetching doi: Applying life cycle assessment to assess the environmental performance of decentralised versus centralised wastewater systems
Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: http://www.jstor.org/stable/25042837
Error fetching doi: https://www.jstor.org/stable/25038362
Error fetching d

In [13]:
"""
doi2bib --input references.txt --output refs.bib 
"""

#read file
with open(cwd / 'data' / 'refs.bib', 'r') as f:
    references = f.readlines()

filtered = filter(lambda x: x != '\n', references)
filtered = map(lambda x: x.replace('\n', ''), filtered)
filtered = map(lambda x: x.replace(' @', '@' ), filtered)

filtered = list(filtered)

#store list of references to txt
with open(cwd / 'data' / 'refs_2.bib', 'w') as f:
    for item in list(filtered):
        f.write(f"{item}\n")

In [14]:
global_refs = {}
sf_cites = add_cites(sf, global_refs)
o3_cites = add_cites(o3, global_refs)
gac_cites = add_cites(gac, global_refs)
uv_cites = add_cites(uv, global_refs)
uf_cites = add_cites(uf, global_refs)
global_refs

{'1.1.1-Tricloroetà': {'Ormad2008-bd'},
 'Àcid perfluoro-octanosulfònic': {'Kruithof2007-os', 'Marcelino2017-wn'},
 'Alaclor': {'RAHMANI201882'},
 'Aldrin': {'RAHMANI201882'},
 'Amoxicilina': {'Rizzo2019-us', 'Scheideler2011-pu'},
 'Atrazina': {'RAHMANI201882', 'Shahmahdi2020-ok', 'Tkachenko2020-sk'},
 'Clofenvinfós': {'RAHMANI201882', 'Tkachenko2020-sk'},
 'Clorpirifòs': {'RAHMANI201882'},
 'Dicofol': {'RAHMANI201882'},
 'Dieldrin': {'RAHMANI201882'},
 'Diuron': {'Kruithof2007-os', 'RAHMANI201882'},
 'Endosulfan': {'RAHMANI201882'},
 'Endrin': {'RAHMANI201882'},
 'Heptaclor': {'RAHMANI201882'},
 'Hexaclorobenzè': {'RAHMANI201882'},
 'Isodrin': {'RAHMANI201882'},
 'Isoproturon': {'RAHMANI201882'},
 'Metolaclor': {'RAHMANI201882'},
 'p.p-DDT': {'RAHMANI201882'},
 'Simazina': {'RAHMANI201882'},
 'Sulfametoxazol': {'Javid2020-ch', 'Kruithof2007-os', 'Radjenovic2009-ud'},
 'Terbutilazina': {'RAHMANI201882'},
 'Terbutrina': {'RAHMANI201882'},
 'Tetracloroetilè': {'ARZATE2020110265', 'Salehi

In [15]:
for entry in correct_links:
    bibtex_parsed = bibtexparser.loads(entry['bibtex'])
    id = bibtex_parsed.entries[0]['ID']
    poll = entry['contaminant']
    if poll not in global_refs:
        global_refs[poll] = set()
    global_refs[poll].add(id)

In [16]:
unique_refs = set()
for key in global_refs:
    unique_refs = unique_refs.union(global_refs[key])

#assign each ref a number
ref_dict = {}
for i, ref in enumerate(unique_refs):
    ref_dict[ref] = i + 1

ref_dict

{'Choubert2011-ic': 1,
 'Bradshaw_2015': 2,
 'Skees_2018': 3,
 'Lambropoulou_2017': 4,
 'Kim_2009': 5,
 'Juksu_2019': 6,
 'R_a_G_mez_2011': 7,
 'Vazquez_Roig_2014': 8,
 'Javid_2020': 9,
 'Hijosa_Valsero_2013': 10,
 'Scheideler2011-pu': 11,
 'Chandurvelan_2015': 12,
 'Neu_1974': 13,
 'Cai_2018': 14,
 'Radjenovi__2009': 15,
 'Lin_2021': 16,
 'RAHMANI201882': 17,
 'ter_Laak_2014': 18,
 'Li_2014': 19,
 'Heaney_2015': 20,
 'Kim_2016': 21,
 'Buzier_2006': 22,
 'Gupta_1998': 23,
 'Goldstone_1990': 24,
 'Lebrun_2015': 25,
 'ARZATE2020110265': 26,
 'Tewari_2013': 27,
 'Choubert_2011': 28,
 'Nasuhoglu_2011': 29,
 'Al_Aukidy_2012': 30,
 'Stoveland_1979': 31,
 'Pagliaccia_2022': 32,
 'Moriyama_1988': 33,
 'Johnson_2017': 34,
 'ISHIWATA_2002': 35,
 'Sterritt_1981': 36,
 'Samarghandi_2007': 37,
 'Kim_2018': 38,
 'Balogh_2008': 39,
 'Javid2020-ch': 40,
 'Shammay_2018': 41,
 'Mahmoudkhani_2014': 42,
 'Novovic_2015': 43,
 'Salehi2020-aa': 44,
 'Woznicki_2015': 45,
 'Marug_n_2020': 46,
 'Shahmahdi_2020'

In [17]:
global_refs_id = {}

for key in global_refs:
    global_refs_id[key] = set()
    for ref in global_refs[key]:
        global_refs_id[key].add(ref_dict[ref])

global_refs_id

{'1.1.1-Tricloroetà': {51, 54, 80, 211},
 'Àcid perfluoro-octanosulfònic': {87, 124, 197},
 'Alaclor': {17, 84, 135, 168},
 'Aldrin': {17, 84, 119},
 'Amoxicilina': {11, 120},
 'Atrazina': {10, 17, 58, 71, 82, 84, 111, 131, 135, 147, 155, 168, 216},
 'Clofenvinfós': {10, 17, 71, 84, 168},
 'Clorpirifòs': {17, 84, 168},
 'Dicofol': {17, 84},
 'Dieldrin': {17, 84, 119, 155},
 'Diuron': {17, 58, 84, 124, 131, 155, 170},
 'Endosulfan': {17, 84, 119},
 'Endrin': {17, 84, 119},
 'Heptaclor': {17, 84, 119},
 'Hexaclorobenzè': {17},
 'Isodrin': {17, 84},
 'Isoproturon': {17, 84, 131, 135, 155, 168},
 'Metolaclor': {17, 84, 107},
 'p.p-DDT': {17, 84, 119},
 'Simazina': {17, 84, 131, 135, 155},
 'Sulfametoxazol': {2,
  5,
  12,
  14,
  18,
  27,
  29,
  30,
  34,
  40,
  45,
  46,
  47,
  61,
  74,
  78,
  79,
  82,
  87,
  92,
  102,
  103,
  112,
  117,
  124,
  127,
  131,
  134,
  137,
  138,
  143,
  144,
  146,
  149,
  152,
  153,
  157,
  159,
  160,
  162,
  166,
  169,
  170,
  171,
  

In [41]:
references_dict = {}
for pollutant in global_refs:
    references_pollutant = []
    for key in global_refs[pollutant]:
        references_pollutant.append(f"\\cite{{{key}}}")

    references_dict[pollutant] = ",".join(references_pollutant)


In [43]:
merged = pd.merge(atenuacio_df, traduccio_df, how='left', left_on='contaminant', right_on='Component').dropna().drop(columns=['Component'])
merged = merged.set_index('contaminant')
new_d = pd.Series(references_dict)
merged['References'] = new_d
merged.set_index('Component en', inplace=True)
merged['Tipus'] = merged['Tipus'].replace('preferent', 'preferential')
merged['Tipus'] = merged['Tipus'].replace('prioritària', 'priority')

#round all to 2 decimals
merged = merged.round(2).astype(str)


merged

,UV,CL,SF,UF,GAC,RO,AOP,O3,OTHER,Primari,C,CN,CNP,coef,Tipus,References
Component en,,,,,,,,,,,,,,,,
"1,1,1-Trichloroethane",0.0,0.0,0,10.0,0.0,0.0,0,0.0,0,17.4,97.4,97.4,97.4,0.0,preferential,"\cite{Ormad2008-bd},\cite{Bhattacharya_1996},\..."
"1,2-Dichloroethane",0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,24.0,70.0,70.0,70.0,0.0,priority,nan
Di (2-ethylhexyl) phthalate (DEHP),0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,63.0,75.0,82.0,82.0,0.0,priority,\cite{Kim2009-ik}
Alachlor,5.38,5.0,0,0.0,0.0,0.0,0,0.0,0,0.0,65.0,65.0,65.0,0.0,priority,"\cite{Ormad_2008},\cite{RAHMANI201882},\cite{G..."
Aldrin,0.0,5.0,0,0.0,0.0,0.0,0,0.0,0,53.5,27.8,27.6,27.6,0.0,priority,"\cite{Ormad_2008},\cite{RAHMANI201882},\cite{K..."
Anthracene,7.95,0.0,0,0.0,0.0,0.0,0,0.0,0,26.75,18.57,20.04,20.04,0.0,priority,\cite{Teijon_2010}
Arsenic,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0,73.84,34.01,49.62,49.62,0.0,preferential,"\cite{Feng_2018},\cite{Harper_1992},\cite{Droz..."
Atrazine,7.53,5.0,0,10.0,0.0,0.0,0,0.0,0,0.0,53.0,53.0,53.0,0.0,priority,"\cite{Teijon_2010},\cite{Mailler_2013},\cite{S..."
Benzo(b) fluoride,0.0,0.0,0,10.0,0.0,0.0,0,0.0,0,34.48,19.29,19.2,19.29,0.0,priority,\cite{Teijon_2010}


In [44]:
merged.rename(index={"p.p'-DTT, clofenotane, chlorophnothane": "p.p'-DTT", "Di (2-ethylhexyl) phthalate (DEHP)": "DEHP"}, inplace=True)

In [45]:
merged_1 = merged.drop(columns=['References'])
merged_1 = merged_1.rename(columns={'Primari': 'P', 'Tipus': 'Type'})
merged_2 = merged[['References']]

In [ ]:
#convert styler to pandas


In [52]:
merged_1.style.to_latex(cwd / 'data' / 'generation_attenuation.tex', clines="all;data")
merged_2.style.to_latex(cwd / 'data' / 'generation_attenuation_2.tex', clines="all;data")

In [20]:
footnote_list =  []
for key in ref_dict:
    id = ref_dict[key]
    footnote_list.append(f"$^{{{id}}}$ \\cite{{{key}}}")

footnote = ', '.join(footnote_list)
footnote = '\\footnotesize{' + footnote + '}\\'

In [21]:
footnote

'\\footnotesize{$^{1}$ \\cite{Choubert2011-ic}, $^{2}$ \\cite{Bradshaw_2015}, $^{3}$ \\cite{Skees_2018}, $^{4}$ \\cite{Lambropoulou_2017}, $^{5}$ \\cite{Kim_2009}, $^{6}$ \\cite{Juksu_2019}, $^{7}$ \\cite{R_a_G_mez_2011}, $^{8}$ \\cite{Vazquez_Roig_2014}, $^{9}$ \\cite{Javid_2020}, $^{10}$ \\cite{Hijosa_Valsero_2013}, $^{11}$ \\cite{Scheideler2011-pu}, $^{12}$ \\cite{Chandurvelan_2015}, $^{13}$ \\cite{Neu_1974}, $^{14}$ \\cite{Cai_2018}, $^{15}$ \\cite{Radjenovi__2009}, $^{16}$ \\cite{Lin_2021}, $^{17}$ \\cite{RAHMANI201882}, $^{18}$ \\cite{ter_Laak_2014}, $^{19}$ \\cite{Li_2014}, $^{20}$ \\cite{Heaney_2015}, $^{21}$ \\cite{Kim_2016}, $^{22}$ \\cite{Buzier_2006}, $^{23}$ \\cite{Gupta_1998}, $^{24}$ \\cite{Goldstone_1990}, $^{25}$ \\cite{Lebrun_2015}, $^{26}$ \\cite{ARZATE2020110265}, $^{27}$ \\cite{Tewari_2013}, $^{28}$ \\cite{Choubert_2011}, $^{29}$ \\cite{Nasuhoglu_2011}, $^{30}$ \\cite{Al_Aukidy_2012}, $^{31}$ \\cite{Stoveland_1979}, $^{32}$ \\cite{Pagliaccia_2022}, $^{33}$ \\cite{M

In [22]:
#store in txt

with open(cwd / 'data' / 'footnote.txt', 'w') as f:
    f.write(footnote)

In [27]:
compound_features = pd.read_excel(cwd / 'data' / 'compound_features.xlsx')
compound_features_merged = pd.merge(traduccio_df, compound_features, how='left', left_on='Component', right_on='name')
compound_features_merged = compound_features_merged.drop(columns=['Component', 'Tipus', 'name', 'kow']).set_index('Component en')
compound_features_merged = compound_features_merged.round(2).dropna().astype(str)
compound_features_merged

,solub,aq_hlife,aq_volat,mol_wt,aq_resus,aq_settle,ben_act_dep,ben_bury,ben_hlife,log(kow)
Component en,,,,,,,,,,
"1,1,1-Trichloroethane",666.1,0.19,0.0,133.4,0.0,0.5,0.3,0.0,0.19,2.68
"1,2-Dichloroethane",6414.0,0.58,0.0,98.96,0.0,0.5,0.3,0.0,0.58,1.83
Alachlor,18.07,0.66,0.0,269.77,0.0,0.5,0.3,0.0,0.66,3.37
Aldrin,0.01,2.13,0.0,364.91,0.0,0.5,0.3,0.0,2.13,6.75
Amoxicillin,3433.0,0.54,0.0,365.4,0.0,0.5,0.3,0.0,0.54,0.97
...,...,...,...,...,...,...,...,...,...,...
Trichlorethylene,778.7,0.36,0.0,131.39,0.0,0.5,0.3,0.0,0.36,2.47
Trichloromethane,2096.0,1.08,0.0,119.38,0.0,0.5,0.3,0.0,1.08,1.52
Trifluralin,12.2,0.92,0.0,407.5,0.0,0.5,0.3,0.0,0.0,5.31


In [28]:
compound_features_merged.style.to_latex(cwd / 'data' / 'compound_features.tex', clines="all;data")